In [37]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [38]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

In [39]:
df_cities_skylines = spark \
    .read \
        .parquet('file:///home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/assunto_cities_skylines/extracao_data_2023_10_15/estatisticas/historico_video_tratada.parquet')

In [40]:
df_cities_skylines = df_cities_skylines.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd"))

df_cities_skylines.show(truncate=False)

+-------------+-------------+----------------------+--------------------------+------------------------+------------+-----------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
df_cities_skylines.columns

['data_extracao',
 'ID_VIDEO',
 'DATA_PUBLICACAO',
 'ID_CANAL',
 'NM_CANAL',
 'ID_CATEGORIA',
 'TITULO_VIDEO',
 'DESCRICAO',
 'TAGS',
 'DURACAO_VIDEOS',
 'TOTAL_VISUALIZACOES',
 'TOTAL_LIKES',
 'TOTAL_FAVORITOS',
 'TOTAL_COMENTARIOS']

In [64]:

df_cities_skylines.select(
        'data_extracao',
        f.explode('DATA_PUBLICACAO').alias('DATA_PUBLICACAO'),
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                f.explode('ID_CANAL').alias('ID_CANAL'),
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                f.explode('NM_CANAL').alias('NM_CANAL'), 
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                f.explode('ID_CATEGORIA').alias('ID_CATEGORIA'),
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                f.explode('TITULO_VIDEO').alias('TITULO_VIDEO'),
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                f.explode('DESCRICAO').alias('DESCRICAO'),
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'DESCRICAO',
                f.explode('TAGS').alias('TAGS'),
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                f.explode('DURACAO_VIDEOS').alias('DURACAO_VIDEOS'),
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                f.explode('TOTAL_VISUALIZACOES').alias('TOTAL_VISUALIZACOES'),
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                f.explode('TOTAL_LIKES').alias('TOTAL_LIKES'),
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                f.explode('TOTAL_FAVORITOS').alias('TOTAL_FAVORITOS'),
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                f.explode('TOTAL_COMENTARIOS').alias('TOTAL_COMENTARIOS'),
        ).show(truncate=False)

+-------------+--------------------+------------------------+----------------------+------------+---------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
d

+-------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+--------------------+
|data_extracao|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|            ID_CANAL|
+-------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+--------------------+
|   2023-10-15|[UCTrNnXivZU_1-aJ...|            [DILSÃO]|        [20]|[Começando do Zer...|[sejam bem vindos...|[[cities, cities ...|         [P0D]|               [13]|        [2]|            [0]|              [0]|UCTrNnXivZU_1-aJq...|
|   2023-10-15|[UCmg3_taw1BrGuo1...|      [O Construtor]

In [56]:
df_cities_skylines.select('data_extracao').distinct().show()

+-------------+
|data_extracao|
+-------------+
|   2023-10-26|
|   2023-10-25|
|   2023-10-23|
|   2023-10-24|
|   2023-10-22|
|   2023-10-21|
|   2023-10-19|
|   2023-10-20|
|   2023-10-18|
|   2023-10-17|
|   2023-10-15|
|   2023-10-16|
+-------------+



In [65]:
f_cities_skylines_comentarios = spark \
    .read \
        .parquet('file:///home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/assunto_cities_skylines/extracao_data_2023_10_15/comentarios/historico_comentario_tratada.parquet')

In [66]:
f_cities_skylines_comentarios.show()

+-------------------+--------------------+--------------------+-----------+--------------------+-----------+--------------------+--------------------+---------------+
|      DATA_EXTRACAO|            ID_CANAL|       ID_COMENTARIO|   ID_VIDEO|    TEXTO_COMENTARIO|TOTAL_LIKES|     DATA_PUBLICACAO|    DATA_ATUALIZACAO|TOTAL_RESPOSTAS|
+-------------------+--------------------+--------------------+-----------+--------------------+-----------+--------------------+--------------------+---------------+
|2023-10-15 14:50:36|UCrOH1V-FyMunBIMr...|Ugx3JrNgDMhHKRW1s...|XOUzWJ0bHuw|Eu amo essa mecân...|          4|2023-10-15T16:13:16Z|2023-10-15T16:13:16Z|              0|
|2023-10-15 14:50:36|UCrOH1V-FyMunBIMr...|Ugy7JISBBIETScjkR...|XOUzWJ0bHuw|                  😮|          0|2023-10-15T15:57:50Z|2023-10-15T15:57:50Z|              0|
|2023-10-15 14:50:36|UCrOH1V-FyMunBIMr...|Ugwdyu3lBkkFmvB6P...|XOUzWJ0bHuw|           Promissor|          0|2023-10-15T15:55:36Z|2023-10-15T15:55:36Z|              0|

In [ ]:
f_cities_skylines_comentarios = spark \
    .read \
        .parquet('file:///home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/assunto_cities_skylines/extracao_data_2023_10_15/resposta_comentarios/historico_resposta_comentarios.parquet/')

In [67]:
f_cities_skylines_comentarios.show()

+-------------------+--------------------+--------------------+-----------+--------------------+-----------+--------------------+--------------------+---------------+
|      DATA_EXTRACAO|            ID_CANAL|       ID_COMENTARIO|   ID_VIDEO|    TEXTO_COMENTARIO|TOTAL_LIKES|     DATA_PUBLICACAO|    DATA_ATUALIZACAO|TOTAL_RESPOSTAS|
+-------------------+--------------------+--------------------+-----------+--------------------+-----------+--------------------+--------------------+---------------+
|2023-10-15 14:50:36|UCrOH1V-FyMunBIMr...|Ugx3JrNgDMhHKRW1s...|XOUzWJ0bHuw|Eu amo essa mecân...|          4|2023-10-15T16:13:16Z|2023-10-15T16:13:16Z|              0|
|2023-10-15 14:50:36|UCrOH1V-FyMunBIMr...|Ugy7JISBBIETScjkR...|XOUzWJ0bHuw|                  😮|          0|2023-10-15T15:57:50Z|2023-10-15T15:57:50Z|              0|
|2023-10-15 14:50:36|UCrOH1V-FyMunBIMr...|Ugwdyu3lBkkFmvB6P...|XOUzWJ0bHuw|           Promissor|          0|2023-10-15T15:55:36Z|2023-10-15T15:55:36Z|              0|